In [16]:
import pandas as pd
import numpy as np
from app.features.tabular import engineer_tab_features
# Se necessário, inclua a definição da sua função aqui ou importe-a
# from seu_modulo.tabular import engineer_tab_features

# 1. Crie um DataFrame de exemplo com dados de latitude e longitude
# (Usaremos pontos fictícios para o teste)
# 2. Defina o caminho para o seu arquivo .gpkg e o nome da camada
urban_areas_file = '../data/geo/ibge_areas_urbanizadas.gpkg'
urban_areas_layer = 'lml_area_densamente_edificada_a'


paths = ['../data/aurita_location_fixed.csv', '../data/jacchus_location_fixed.csv', '../data/penicillata_location_fixed.csv']
df = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)

In [17]:
df_tab, tab_cols = engineer_tab_features(
    df, 
    mode="latlon_time", # Ou "latlon_time"
    urban_areas_path=urban_areas_file, 
    urban_layer=urban_areas_layer,
    urban_radius_km=5.0
)

In [18]:
df = pd.concat([df_tab, df['tipo']], axis=1)
df.head()

,latitude,longitude,year,month,dayofyear,month_sin,month_cos,doy_sin,doy_cos,in_urban,dist_urban_poly_m,dist_urban_centroid_m,urban_count_within_5km,tipo
0,-22.415519,-43.398463,2005.0,4.0,100.0,0.866025,-0.500000,0.989372,-0.145404,0.0,993.412968,3539.933511,1.0,N-H
1,-21.925034,-42.871454,2006.0,5.0,121.0,0.500000,-0.866025,0.874481,-0.485060,0.0,8883.343071,9955.594631,1.0,N-H
2,-22.273861,-47.564140,2009.0,10.0,296.0,-0.866025,0.500000,-0.932655,0.360771,0.0,1366.662544,2074.617925,1.0,N-H
3,-21.868276,-42.896158,2011.0,5.0,141.0,0.500000,-0.866025,0.660152,-0.751132,0.0,10083.009568,12101.489233,1.0,N-H
4,-23.159688,-44.820074,2011.0,7.0,182.0,-0.500000,-0.866025,0.017166,-0.999853,0.0,9924.705206,10228.703186,1.0,N-H


In [27]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler

def analisar_correlacao_spearman(df, numeric_col, categorical_col, n_permutations=10000):
    """
    Normaliza a coluna numérica, calcula a correlação de Spearman e 
    realiza um teste de permutação para verificar a significância.

    Args:
        df (pd.DataFrame): O dataframe de entrada.
        numeric_col (str): O nome da coluna numérica.
        categorical_col (str): O nome da coluna categórica.
        n_permutations (int): O número de permutações para o teste.
    """
    print(f"--- Análise de Correlação entre '{numeric_col}' e '{categorical_col}' ---")

    # 1. Normalização da coluna numérica
    scaler = MinMaxScaler()
    x_norm = scaler.fit_transform(df[[numeric_col]]).flatten()
    
    # 2. Codificação da coluna categórica e exibição do mapeamento
    tipo_categoria = df[categorical_col].astype('category')
    y_codes = tipo_categoria.cat.codes.values
    categories = tipo_categoria.cat.categories
    
    print("Mapeamento das categorias para códigos:")
    for code, category in enumerate(categories):
        print(f"'{category}' -> {code}")

    # 3. Cálculo da correlação observada
    corr_obs, _ = spearmanr(x_norm, y_codes)
    
    # 4. Teste de Permutação
    perm_corrs = []
    for _ in range(n_permutations):
        # Embaralha a variável categórica para quebrar a associação
        y_perm = np.random.permutation(y_codes)
        corr_perm, _ = spearmanr(x_norm, y_perm)
        perm_corrs.append(corr_perm)
    
    # 5. Cálculo do valor-p
    perm_corrs = np.array(perm_corrs)
    p_value = (np.abs(perm_corrs) >= np.abs(corr_obs)).mean()
    
    # 6. Impressão dos resultados
    print(f"\nCorrelação de Spearman observada: {corr_obs:.4f}")
    print(f"p-value: {p_value:.4f}")
    
    # 7. Decisão sobre a significância
    if p_value < 0.05:
        print("\nResultado: A correlação é estatisticamente **significativa**.")
        print("Isso sugere que a relação observada não é devido ao acaso.")
    else:
        print("\nResultado: A correlação é **não significativa**.")
        print("Não há evidências suficientes para dizer que a correlação é real.")
    print("\n" + "="*50 + "\n")


# --- Exemplos de como usar a função ---
# Analisar a correlação entre 'latitude' e 'tipo'
analisar_correlacao_spearman(df, 'latitude', 'tipo')

# Analisar a correlação entre 'longitude' e 'tipo'
analisar_correlacao_spearman(df, 'longitude', 'tipo')

--- Análise de Correlação entre 'latitude' e 'tipo' ---
Mapeamento das categorias para códigos:
'H' -> 0
'N-H' -> 1

Correlação de Spearman observada: 0.1025
p-value: 0.0000

Resultado: A correlação é estatisticamente **significativa**.
Isso sugere que a relação observada não é devido ao acaso.


--- Análise de Correlação entre 'longitude' e 'tipo' ---
Mapeamento das categorias para códigos:
'H' -> 0
'N-H' -> 1

Correlação de Spearman observada: -0.0867
p-value: 0.0000

Resultado: A correlação é estatisticamente **significativa**.
Isso sugere que a relação observada não é devido ao acaso.




In [28]:
analisar_correlacao_spearman(df, 'dist_urban_centroid_m', 'tipo')
analisar_correlacao_spearman(df, 'dist_urban_poly_m', 'tipo')
analisar_correlacao_spearman(df, 'in_urban', 'tipo')

--- Análise de Correlação entre 'dist_urban_centroid_m' e 'tipo' ---
Mapeamento das categorias para códigos:
'H' -> 0
'N-H' -> 1

Correlação de Spearman observada: -0.1415
p-value: 0.0000

Resultado: A correlação é estatisticamente **significativa**.
Isso sugere que a relação observada não é devido ao acaso.


--- Análise de Correlação entre 'dist_urban_poly_m' e 'tipo' ---
Mapeamento das categorias para códigos:
'H' -> 0
'N-H' -> 1

Correlação de Spearman observada: 0.0309
p-value: 0.1150

Resultado: A correlação é **não significativa**.
Não há evidências suficientes para dizer que a correlação é real.


--- Análise de Correlação entre 'in_urban' e 'tipo' ---
Mapeamento das categorias para códigos:
'H' -> 0
'N-H' -> 1

Correlação de Spearman observada: -0.0256
p-value: 0.1968

Resultado: A correlação é **não significativa**.
Não há evidências suficientes para dizer que a correlação é real.


